# Setup and Imports

Basic setup for seismic time series analysis:
- matplotlib: For plotting (inline mode)
- numpy: For numerical operations
- obspy: For seismic data handling
  - UTCDateTime: Time handling
  - read: Data reading
  - Stream: Seismic data container
- obspy.signal.filter: For envelope calculation
- pandas: For time series handling
- glob: For file path pattern matching

In [10]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
import numpy as np
from obspy import UTCDateTime, read, Stream
import pandas as pd
import glob
import datetime
from obspy.signal.filter import envelope
import matplotlib.cm as cm

# Data File Selection

Locates and counts seismic data files:
1. File pattern: 'DATA/SDS/2024/8N/HB04/EHZ.D/*'
   - Network: 8N
   - Station: HB04
   - Channel: EHZ (Vertical high-frequency channel)
   - Data structure: SDS (SeisComP Data Structure)
2. Files are sorted chronologically
3. Count available files using len()

In [21]:
# Path to where you have copied the DATA/ folder (that contains the SDS, RESP etc folders)
DATA_PATH = "DATA"

In [22]:
files = sorted(glob.glob(os.path.join(DATA_PATH, 'SDS', '2024', '8N', 'HB04', 'EHZ.D', '*')))

In [23]:
len(files)

31

# Load and Preprocess Seismic Data

Loads and processes seismic data files:
- Iterates through files (skipping first file)
- For each file:
  1. Reads data using ObsPy
  2. Decimates by factor of 5 (reduces sampling rate)
  3. Adds to traces list
- Finally merges all traces into single Stream object

Parameters:
- decimati10 (we have enough with 20 Hz data for now)n factor: 5
- filter: Applied during decimation (no_filter=False)

In [ ]:
traces = []

for file in files:
    print(file)
    try:
        st = read(file)
    except:
        print(f"Error reading {file}")
        continue
    st.decimate(10, no_filter=False)
    traces.append(st[0])
    del(st)
st = Stream(traces=traces)
st.merge()


# Generate Daily Seismic Plot

Creates a publication-quality day plot of seismic data:
Parameters:
- interval: 60 minutes
- format: PDF output
- style settings:
  - Color: black
  - Line width: 0.5
  - DPI: 100
  - Size: 5000x10000
  - Time format: "YYYY-MM-DD HHh"
- Layout adjustments for margins
- Output file: "1hlines.pdf"

In [ ]:
outfile = "1hlines.pdf"
st.plot(type='dayplot', interval=60, right_vertical_labels=False,
        one_tick_per_line=True,
        color=['black'], linewidth=0.5, show_y_UTC_label=False, 
         dpi=100, size=(5000,10000), y_labels_size=10, tick_format="%Y-%m-%d %Hh",
        subplots_adjust_left=0.03, subplots_adjust_right=0.98, subplots_adjust_top=0.98, 
        subplots_adjust_bottom=0.02, outfile=outfile)


# Prepare Data for Overview Visualization

Two-step data preparation:
1. Copy and merge stream:
   - Interpolates gaps
   - Further decimates by factor of 10
2. Data transformation:
   - Convert to absolute values
   - Clip values between 0 and 1e5
   - Reduces noise and extreme values

In [ ]:
st2 = st.copy()
st2 = st2.merge(method=1, fill_value="interpolate")
st2.decimate(10, no_filter=True)

In [ ]:
st2[0].data = np.abs(st2[0].data)
st2[0].data = np.clip(st2[0].data, 0, 1e5)

# Create Overview Plot with Weekend Highlighting

Generates comprehensive time series visualization:
Parameters:
- Figure size: 30x20 inches
- Plot features:
  - Black line for seismic data (width 0.5)
  - Green shaded weekends (Saturday-Monday)
  - Daily tick marks with dates
  - Automatic date formatting
- Output: "groundmotion_continuous_with_weekends.pdf"

This plot provides a complete overview of amplitudemic activity with weekend periods highlighted for context.

In [ ]:
fig = plt.figure(figsize=(30,20))
tr = st2[0]
plt.plot(tr.times("matplotlib"), tr.data, c='k', lw=0.5)
dr = pd.date_range(tr.stats.starttime.date, tr.stats.endtime.date, freq="W-SAT")
for d in dr:
    plt.axvspan(d, d+datetime.timedelta(days=2), zorder=-1, alpha=0.5, ec="none", fc="lightgreen")

dr = pd.date_range(tr.stats.starttime.date, tr.stats.endtime.date, freq="D")
plt.xticks(list(dr), [d.strftime("%Y-%m-%d") for d in list(dr)])

plt.margins(0)

plt.ylim(0,1e4)
fig.autofmt_xdate()
plt.savefig("groundmotion_continuous_with_weekends.pdf")